In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def warn(*args, **kwargs): pass
import warnings
warnings.warn = warn

from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import StratifiedShuffleSplit


In [2]:
train_1 = pd.read_csv(r"C:\Users\piush\Desktop\Dataset\AibHack\data\training.txt", sep="\t", encoding = 'iso-8859-1')
test_1 = pd.read_csv(r"C:\Users\piush\Desktop\Dataset\AibHack\data\testing.txt", sep="\t", encoding = 'iso-8859-1')

In [4]:
def is_outlier(points, thresh = 3.5):
    if len(points.shape) == 1:
        points = points[:,None]
    median = np.median(points, axis=0)
    diff = np.sum((points - median)**2, axis=-1)
    diff = np.sqrt(diff)
    med_abs_deviation = np.median(diff)

    modified_z_score = 0.6745 * diff / med_abs_deviation

    return modified_z_score > thresh

In [5]:
train_1 = train_1[pd.notnull(train_1['EnergyRatingCat'])]

In [6]:
df_train = train_1.ix[:, train_1.columns != 'EnergyRatingCat']

In [7]:
df = df_train.append(test_1, ignore_index = True)

In [8]:
cats = []
for col in df.columns.values:
    if df[col].dtype == 'object':
        cats.append(col)

In [9]:
df_cont = df.drop(cats, axis=1)
df_cat = df[cats]

In [10]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from scipy.stats import skew

for col in df_cont.columns.values:
    if np.sum(df_cont[col].isnull()) > 50:
        #print("Removing Column: {}".format(col))
        df_cont = df_cont.drop(col, axis = 1)
    elif np.sum(df_cont[col].isnull()) > 0:
        #print("Replacing with Median: {}".format(col))
        median = df_cont[col].median()
        idx = np.where(df_cont[col].isnull())[0]
        df_cont[col].iloc[idx] = median
        
        
        outliers = np.where(is_outlier(df_cont[col]))
        df_cont[col].iloc[outliers] = median
        
               
        if skew(df_cont[col]) > 0.75:
            #print("Skewness Detected: {}".format(col))
            df_cont[col] = np.log(df_cont[col])
            df_cont[col] = df_cont[col].apply(lambda x: 0 if x == -np.inf else x)
        
        df_cont[col] = Normalizer().fit_transform(df_cont[col].reshape(1,-1))[0]

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:22: RuntimeWarning: divide by zero encountered in log
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: RuntimeWarning: divide by zero encountered in true_divide
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: RuntimeWarning: invalid value encountered in true_divide
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:11: RuntimeWarning: invalid value encountered in greater


In [11]:
target = train_1['EnergyRatingCat']

In [12]:
target = train_1[pd.notnull(train_1['EnergyRatingCat'])]
target = train_1['EnergyRatingCat']
target = target.dropna()
target = target.values

In [15]:
target = train_1['EnergyRatingCat'].replace({"Nan" : -1, "A1" : 15 , "A3" : 13, "A2" : 14,"B1" : 12, "B2" : 11, "B3" : 10, "C1" : 9 , "C2" : 8, "C3" : 7,"D1" : 6, "D2" : 5,"D3" : 4, "E1" : 3 , "E2" : 2 , "F " : 1 ,"G " : 0})

In [16]:
target.isnull().values.any()

False

In [17]:
train = df_cont[:train_1.shape[0]]
test = df_cont[train_1.shape[0]:]

In [18]:
print (str(len(train))+" rows for training set")
print (str(len(test))+" rows for test set")
print (str(len(target))+" rows for test set")

464502 rows for training set
159262 rows for test set
464502 rows for test set


In [29]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100)
clf.fit(train, target)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [ ]:
test_y = clf.predict(test)

In [26]:
test_y = test_y.replace({-1 : "Nan", 15 : "A1"  , 13 : "A3" , 14 : "A2" ,12 : "B1", 11 : "B2", 10 : "B3", 9 : test_y = test_y.replace({-1 : "Nan", 15 : "A1"  , 13 : "A3" , 14 : "A2" ,12 : "B1", 11 : "B2", 10 : "B3", 9 : test_y = test_y.replace({-1 : "Nan", 15 : "A1"  , 13 : "A3" , 14 : "A2" ,12 : "B1", 11 : "B2", 10 : "B3", 9 : "C1", 8 : "C2", 7 : "C3",6 : "D1", 5 : "D2", 4 : "D3", "E1" : 3 , "E2" : 2 , "F " : 1 ,"G " : 0}), "C2" : 8, "C3" : 7,"D1" : 6, "D2" : 5,"D3" : 4, "E1" : 3 , "E2" : 2 , "F " : 1 ,"G " : 0}), "C2" : 8, "C3" : 7,"D1" : 6, "D2" : 5,"D3" : 4, "E1" : 3 , "E2" : 2 , "F " : 1 ,"G " : 0})

array([6, 6, 6, ..., 6, 6, 6], dtype=int64)

In [27]:
submission = pd.DataFrame()
submission["BuildingID"] = test.index
submission["EnergyRatingCat"]= test_y

submission.to_csv("final_result.txt",index=False,sep ="\t")